<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250917%5D%20Dataset-Verified%20IDS%20Domain-Shift%20Pipeline%20(CIC_IoMT_2024_WiFi_MQTT%20%2B%20NF-ToN-IoT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset-Verified IDS Domain-Shift Pipeline (CIC_IoMT_2024_WiFi_MQTT + NF-ToN-IoT)
- Uses actual columns detected from the provided files
- Builds class-family mapping from string labels (e.g., "label", "Attack")
- Implements MQTT filtering on NF-ToN-IoT via L4_DST_PORT
- Trains ablations: LR-only, BiLSTM-only, LR->BiLSTM (+ optional adversarial mixing)
- Reports Macro-F1, per-class Recall, AUROC/PR-AUC, calibration (ECE/MCE), and FPR@1e-3/1e-4

Author: Sine & Mentor

In [48]:
import os
import sys
import math
import time
import warnings
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.metrics import (f1_score, recall_score, roc_auc_score, average_precision_score,
                             roc_curve)
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight

from imblearn.combine import SMOTETomek

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.utils import to_categorical

warnings.filterwarnings("ignore")

In [49]:
# ----------------------------
# Config (set to your files)
# ----------------------------
CONFIG = {
    "NF_TON_IOT_CSV": "Dataset_NF-ToN-IoT.csv",                 # provided
    "CIC_IOMT_TRAIN_CSV": "CIC_IoMT_2024_WiFi_MQTT_train.csv",  # provided
    "CIC_IOMT_TEST_CSV":  "CIC_IoMT_2024_WiFi_MQTT_test.csv",   # provided
    # Column names (verified)
    "NF_NUMERIC_CLASS_COL": "Class",      # 0..9 (0=benign)
    "NF_TEXT_LABEL_COL": "Attack",        # string names: 'Benign','ddos','dos','scanning',...
    "CIC_CLASS_COL": "Class",             # numeric id in both train/test
    "CIC_TEXT_LABEL_COL": "label",        # string like 'TCP_IP-DDoS-UDP2_train'
    # Port column for MQTT filtering
    "NF_DST_PORT_COL": "L4_DST_PORT",
    "MQTT_PORTS": [1883, 8883],
    # Experiment toggles
    "SEED": 42,
    "EPOCHS": 30,
    "BATCH_SIZE": 128,
    "LEARNING_RATE": 1e-3,
    "PATIENCE": 5,
    "RESAMPLING": "none",                 # "none" | "smote_tomek"
    "LOSS_MODE": "class_balanced_ce",     # "ce" | "class_balanced_ce" | "focal"
    "USE_ADV_TRAINING": True,
    "ADV_METHOD": "fgsm",                 # "fgsm" | "pgd"
    "FGSM_EPS": 0.05,
    "PGD_EPS": 0.03,
    "PGD_STEPS": 5,
    "PGD_ALPHA": 0.01,
    "ADV_RATIO": 0.5,
    "OUTDIR": "./outputs"
}

np.random.seed(CONFIG["SEED"])
tf.random.set_seed(CONFIG["SEED"])
os.makedirs(CONFIG["OUTDIR"], exist_ok=True)

In [50]:
# ----------------------------
# Family mapping from strings
# ----------------------------
FAMILIES = ["Benign", "DoS_DDoS", "Recon_Scan", "MQTT", "Spoof", "Other"]

def map_label_string_to_family(s: str) -> str:
    if not isinstance(s, str):
        return "Other"
    st = s.lower()
    if "benign" in st or st == "normal":
        return "Benign"
    if "mqtt" in st:
        return "MQTT"
    if "ddos" in st or "dos" in st:
        return "DoS_DDoS"
    if "scan" in st or "recon" in st or "portscan" in st:
        return "Recon_Scan"
    if "spoof" in st or "mitm" in st or "impersonat" in st or "man-in-the-middle" in st:
        return "Spoof"
    return "Other"

In [51]:
# ----------------------------
# Loading & preprocessing
# ----------------------------
def load_nf_ton_iot(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    return df

def load_cic_iomt(train_path: str, test_path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    dtr = pd.read_csv(train_path)
    dte = pd.read_csv(test_path)
    return dtr, dte

def mqtt_filter_nf(df: pd.DataFrame, dst_port_col: str, ports: List[int]) -> pd.DataFrame:
    if dst_port_col not in df.columns:
        print(f"[WARN] {dst_port_col} not in columns; skip MQTT filter.")
        return df
    try:
        f = df[df[dst_port_col].astype(int).isin(ports)].copy()
    except Exception:
        f = df[df[dst_port_col].astype(str).isin(list(map(str, ports)))].copy()
    if len(f) == 0:
        print("[WARN] MQTT filter produced empty set; reverting to full NF-ToN-IoT.")
        return df
    return f

def encode_and_scale(df: pd.DataFrame, y_col: str) -> Tuple[np.ndarray, np.ndarray, StandardScaler, List[str]]:
    df = df.copy()
    y = df[y_col].values
    Xdf = df.drop(columns=[y_col])
    for c in Xdf.columns:
        if not np.issubdtype(Xdf[c].dtype, np.number):
            Xdf[c] = LabelEncoder().fit_transform(Xdf[c].astype(str))
    Xdf = Xdf.fillna(0)
    scaler = StandardScaler()
    X = scaler.fit_transform(Xdf.values)
    return X, y, scaler, Xdf.columns.tolist()

In [52]:
# ----------------------------
# Feature builders (schema-aligned)
# ----------------------------
def build_features_train(df: pd.DataFrame, y_col: str):
    df = df.copy()
    y = df[y_col].values
    Xdf = df.drop(columns=[y_col])

    # split numeric vs non-numeric
    Xnum = Xdf.select_dtypes(include=[np.number]).copy().fillna(0)
    Xcat = Xdf.select_dtypes(exclude=[np.number]).astype(str)
    if Xcat.shape[1] > 0:
        Xcat = pd.get_dummies(Xcat, dummy_na=False, drop_first=False)
    else:
        Xcat = pd.DataFrame(index=Xdf.index)

    Xall = pd.concat([Xnum, Xcat], axis=1)
    scaler = StandardScaler().fit(Xall.values)
    X = scaler.transform(Xall.values)
    cols = Xall.columns.tolist()
    return X, y, scaler, cols

def build_features_test(df: pd.DataFrame, y_col: str, scaler: StandardScaler, cols_schema: list):
    df = df.copy()
    y = df[y_col].values
    Xdf = df.drop(columns=[y_col])

    Xnum = Xdf.select_dtypes(include=[np.number]).copy().fillna(0)
    Xcat = Xdf.select_dtypes(exclude=[np.number]).astype(str)
    if Xcat.shape[1] > 0:
        Xcat = pd.get_dummies(Xcat, dummy_na=False, drop_first=False)
    else:
        Xcat = pd.DataFrame(index=Xdf.index)

    Xall = pd.concat([Xnum, Xcat], axis=1)

    # align to training schema
    Xall = Xall.reindex(columns=cols_schema, fill_value=0)
    X = scaler.transform(Xall.values)
    return X, y

In [53]:
# ----------------------------
# Resampling
# ----------------------------
def apply_resampling(X: np.ndarray, y: np.ndarray, mode: str):
    if mode == "smote_tomek":
        sampler = SMOTETomek(random_state=CONFIG["SEED"])
        return sampler.fit_resample(X, y)
    return X, y

In [54]:
# ----------------------------
# Loss functions & calibration
# ----------------------------
def focal_loss(gamma=2.0, alpha=None):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0-1e-7)
        ce = -y_true * tf.math.log(y_pred)
        if alpha is not None:
            ce = alpha * ce
        weight = tf.pow(1.0 - y_pred, gamma)
        return tf.reduce_sum(weight * ce, axis=1)
    return loss

def get_loss(num_classes, y_train, mode="ce"):
    if mode == "focal":
        return focal_loss(gamma=2.0), None
    if mode == "class_balanced_ce":
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        cw = {int(c): float(w) for c, w in zip(classes, weights)}
        return tf.keras.losses.CategoricalCrossentropy(), cw
    return tf.keras.losses.CategoricalCrossentropy(), None

def calibration_bins(probs: np.ndarray, y_true: np.ndarray, n_bins: int = 15):
    conf = probs.max(axis=1)
    preds = probs.argmax(axis=1)
    correct = (preds == y_true).astype(int)
    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    mce = 0.0
    rel = []
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        idx = np.where((conf >= lo) & (conf < hi))[0]
        if len(idx) == 0:
            rel.append((0.5*(lo+hi), np.nan, 0))
            continue
        acc = correct[idx].mean()
        conf_mean = conf[idx].mean()
        gap = abs(acc - conf_mean)
        ece += (len(idx) / len(conf)) * gap
        mce = max(mce, gap)
        rel.append((conf_mean, acc, len(idx)))
    return ece, mce, rel

def fpr_at_threshold(y_true_bin: np.ndarray, attack_scores: np.ndarray, target_fpr: float):
    fpr, tpr, thr = roc_curve(y_true_bin, attack_scores)[:3]
    idx = np.where(fpr <= target_fpr)[0]
    if len(idx) == 0:
        j = int(np.argmin(fpr))
        return float(thr[j]), float(fpr[j]), float(tpr[j])
    j = idx[-1]
    return float(thr[j]), float(fpr[j]), float(tpr[j])

In [55]:
# ----------------------------
# Models
# ----------------------------
def build_bilstm(input_shape, num_classes):
    model = models.Sequential([
        layers.Bidirectional(layers.LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(1e-4)), input_shape=input_shape),
        layers.Dropout(0.2),
        layers.Bidirectional(layers.LSTM(32, kernel_regularizer=regularizers.l2(1e-4))),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax", kernel_regularizer=regularizers.l2(1e-4))
    ])
    return model

def fgsm(model, x, y, eps=0.05):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        pred = model(x, training=False)
        loss = tf.keras.losses.categorical_crossentropy(y, pred)
    grad = tape.gradient(loss, x)
    x_adv = x + eps * tf.sign(grad)
    return tf.clip_by_value(x_adv, -10, 10).numpy()

def pgd(model, x, y, eps=0.03, alpha=0.01, steps=5):
    x0 = tf.convert_to_tensor(x, dtype=tf.float32)
    x_adv = tf.identity(x0)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    for _ in range(steps):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            pred = model(x_adv, training=False)
            loss = tf.keras.losses.categorical_crossentropy(y, pred)
        grad = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(grad)
        x_adv = tf.clip_by_value(x_adv, x0 - eps, x0 + eps)
    return x_adv.numpy()

In [56]:
# ----------------------------
# Evaluation
# ----------------------------
@dataclass
class RunResult:
    setting: str
    model_name: str
    use_adv: bool
    resampling: str
    loss_mode: str
    macro_f1: float
    macro_recall: float
    roc_auc_ovr: float
    pr_auc_ovr: float
    ece: float
    mce: float
    fpr1e3: float
    tpr_at_fpr1e3: float
    fpr1e4: float
    tpr_at_fpr1e4: float

def evaluate_multiclass(y_true, proba) -> Tuple[float, float, float, float]:
    classes = np.unique(y_true)
    Yb = label_binarize(y_true, classes=classes)
    try:
        roc = roc_auc_score(Yb, proba, average='macro', multi_class='ovr')
    except Exception:
        roc = float('nan')
    try:
        pr = average_precision_score(Yb, proba, average='macro')
    except Exception:
        pr = float('nan')
    macro_f1 = f1_score(y_true, proba.argmax(axis=1), average='macro')
    macro_rec = recall_score(y_true, proba.argmax(axis=1), average='macro')
    return macro_f1, macro_rec, roc, pr

In [57]:
# ----------------------------
# Main experiment
# ----------------------------

def main():
    # Load raw CSVs
    nf = load_nf_ton_iot(CONFIG["NF_TON_IOT_CSV"])
    cic_tr, cic_te = load_cic_iomt(CONFIG["CIC_IOMT_TRAIN_CSV"], CONFIG["CIC_IOMT_TEST_CSV"])

    # Map strings -> families
    if CONFIG["NF_TEXT_LABEL_COL"] in nf.columns:
        nf_family = nf[CONFIG["NF_TEXT_LABEL_COL"]].astype(str).apply(map_label_string_to_family)
    else:
        nf_family = np.where(nf[CONFIG["NF_NUMERIC_CLASS_COL"]].values == 0, "Benign", "Other")

    cic_tr_family = cic_tr[CONFIG["CIC_TEXT_LABEL_COL"]].astype(str).apply(map_label_string_to_family) if CONFIG["CIC_TEXT_LABEL_COL"] in cic_tr.columns else cic_tr[CONFIG["CIC_CLASS_COL"]].astype(int).apply(lambda _: "Other")
    cic_te_family = cic_te[CONFIG["CIC_TEXT_LABEL_COL"]].astype(str).apply(map_label_string_to_family) if CONFIG["CIC_TEXT_LABEL_COL"] in cic_te.columns else cic_te[CONFIG["CIC_CLASS_COL"]].astype(int).apply(lambda v: "Benign" if v==0 else "Other")

    cic_tr2 = cic_tr.copy(); cic_tr2["Family"] = cic_tr_family.values
    cic_te2 = cic_te.copy(); cic_te2["Family"] = cic_te_family.values

    nf_mqtt = mqtt_filter_nf(nf, CONFIG["NF_DST_PORT_COL"], CONFIG["MQTT_PORTS"])
    if len(nf_mqtt) == 0:
        nf2 = nf.copy()
        nf2["Family"] = nf_family.values
        print("[WARN] MQTT filter empty; using full NF-ToN-IoT.")
    else:
        nf2 = nf_mqtt.copy()
        nf2["Family"] = nf_family.loc[nf_mqtt.index].values

    # Label encoder for target
    fam_le = LabelEncoder().fit(FAMILIES)

    # Build an in-domain split for NF-ToN-IoT once (train/test for IoT→IoT runs)
    y_nf_all_enc = fam_le.transform(nf2["Family"].values)
    nf_train, nf_test = train_test_split(nf2, test_size=0.2, random_state=CONFIG["SEED"],
                                         stratify=y_nf_all_enc)

    results = []
    perclass_dfs = []
    relbin_dfs = []

    def run_setting(train_domain: str, test_domain: str, model_kind: str, use_adv: bool):
        # Choose train/test DataFrames by domain
        if train_domain == "IoMT":
            df_tr = cic_tr2
        else:
            df_tr = nf_train

        if test_domain == "IoMT":
            df_te = cic_te2
        else:
            df_te = nf_test if train_domain == "IoT" and test_domain == "IoT" else nf2

        # Encode targets
        df_tr = df_tr.copy(); df_te = df_te.copy()
        df_tr["y_enc"] = fam_le.transform(df_tr["Family"].values)
        df_te["y_enc"] = fam_le.transform(df_te["Family"].values)

        # Build schema-aligned features using train schema
        Xtr, ytr, scaler, cols_schema = build_features_train(df_tr.drop(columns=["Family"]).rename(columns={"y_enc": "Target"}), y_col="Target")
        Xte, yte = build_features_test(df_te.drop(columns=["Family"]).rename(columns={"y_enc": "Target"}), y_col="Target", scaler=scaler, cols_schema=cols_schema)

        # Optional resampling on train only
        if CONFIG["RESAMPLING"] == "smote_tomek":
            sampler = SMOTETomek(random_state=CONFIG["SEED"])
            Xtr, ytr = sampler.fit_resample(Xtr, ytr)

        # Class weights / loss
        if CONFIG["LOSS_MODE"] == "class_balanced_ce":
            classes = np.unique(ytr)
            weights = compute_class_weight(class_weight='balanced', classes=classes, y=ytr)
            cw = {int(c): float(w) for c, w in zip(classes, weights)}
            loss = tf.keras.losses.CategoricalCrossentropy()
        elif CONFIG["LOSS_MODE"] == "focal":
            cw = None; loss = focal_loss(gamma=2.0)
        else:
            cw = None; loss = tf.keras.losses.CategoricalCrossentropy()

        # Train/eval per model kind
        if model_kind == "LR-only":
            clf = LogisticRegression(max_iter=1000, multi_class='multinomial')
            clf.fit(Xtr, ytr)
            proba_te = clf.predict_proba(Xte)

        elif model_kind == "BiLSTM-only":
            Xtr_seq = Xtr.reshape((Xtr.shape[0], 1, Xtr.shape[1]))
            Xte_seq = Xte.reshape((Xte.shape[0], 1, Xte.shape[1]))
            ytr_oh = to_categorical(ytr, num_classes=len(FAMILIES))
            model = build_bilstm(Xtr_seq.shape[1:], len(FAMILIES))
            model.compile(optimizer=tf.keras.optimizers.Adam(CONFIG["LEARNING_RATE"]), loss=loss, metrics=['accuracy'])
            es = callbacks.EarlyStopping(patience=CONFIG["PATIENCE"], restore_best_weights=True, monitor='val_loss')

            Xtrain_in, ytrain_in = Xtr_seq, ytr_oh
            if use_adv and CONFIG["USE_ADV_TRAINING"]:
                model.fit(Xtr_seq, ytr_oh, validation_split=0.1, epochs=5, batch_size=CONFIG["BATCH_SIZE"], verbose=0)
                X_adv = fgsm(model, Xtr_seq, ytr_oh, eps=CONFIG["FGSM_EPS"]) if CONFIG["ADV_METHOD"]=="fgsm" else pgd(model, Xtr_seq, ytr_oh, eps=CONFIG["PGD_EPS"], alpha=CONFIG["PGD_ALPHA"], steps=CONFIG["PGD_STEPS"])
                k = int(CONFIG["ADV_RATIO"] * Xtr_seq.shape[0]); idx = np.random.choice(Xtr_seq.shape[0], size=k, replace=False)
                Xtrain_in = np.concatenate([Xtr_seq, X_adv[idx]], axis=0); ytrain_in = np.concatenate([ytr_oh, ytr_oh[idx]], axis=0)

            model.fit(Xtrain_in, ytrain_in, validation_split=0.2, epochs=CONFIG["EPOCHS"], batch_size=CONFIG["BATCH_SIZE"], callbacks=[es], verbose=2, class_weight=cw)
            proba_te = model.predict(Xte_seq, batch_size=CONFIG["BATCH_SIZE"], verbose=0)

        elif model_kind == "LR->BiLSTM":
            # LR projection (train on aligned features)
            lr = LogisticRegression(max_iter=1000, multi_class='multinomial')
            lr.fit(Xtr, ytr)
            Xtr_lr = lr.predict_proba(Xtr)
            Xte_lr = lr.predict_proba(Xte)

            Xtr_seq = Xtr_lr.reshape((Xtr_lr.shape[0], 1, Xtr_lr.shape[1]))
            Xte_seq = Xte_lr.reshape((Xte_lr.shape[0], 1, Xte_lr.shape[1]))
            ytr_oh = to_categorical(ytr, num_classes=len(FAMILIES))

            model = build_bilstm(Xtr_seq.shape[1:], len(FAMILIES))
            model.compile(optimizer=tf.keras.optimizers.Adam(CONFIG["LEARNING_RATE"]), loss=loss, metrics=['accuracy'])
            es = callbacks.EarlyStopping(patience=CONFIG["PATIENCE"], restore_best_weights=True, monitor='val_loss')

            Xtrain_in, ytrain_in = Xtr_seq, ytr_oh
            if use_adv and CONFIG["USE_ADV_TRAINING"]:
                model.fit(Xtr_seq, ytr_oh, validation_split=0.1, epochs=5, batch_size=CONFIG["BATCH_SIZE"], verbose=0)
                X_adv = fgsm(model, Xtr_seq, ytr_oh, eps=CONFIG["FGSM_EPS"]) if CONFIG["ADV_METHOD"]=="fgsm" else pgd(model, Xtr_seq, ytr_oh, eps=CONFIG["PGD_EPS"], alpha=CONFIG["PGD_ALPHA"], steps=CONFIG["PGD_STEPS"])
                k = int(CONFIG["ADV_RATIO"] * Xtr_seq.shape[0]); idx = np.random.choice(Xtr_seq.shape[0], size=k, replace=False)
                Xtrain_in = np.concatenate([Xtr_seq, X_adv[idx]], axis=0); ytrain_in = np.concatenate([ytr_oh, ytr_oh[idx]], axis=0)

            model.fit(Xtrain_in, ytrain_in, validation_split=0.2, epochs=CONFIG["EPOCHS"], batch_size=CONFIG["BATCH_SIZE"], callbacks=[es], verbose=2, class_weight=cw)
            proba_te = model.predict(Xte_seq, batch_size=CONFIG["BATCH_SIZE"], verbose=0)
        else:
            raise ValueError("Unknown model_kind")

        # Metrics
        macro_f1, macro_rec, roc_ovr, pr_ovr = evaluate_multiclass(yte, proba_te)
        ece, mce, rel = calibration_bins(proba_te, yte, n_bins=15)

        # Low-FPR metrics (binary collapse)
        benign_idx = FAMILIES.index("Benign")
        attack_scores = 1.0 - proba_te[:, benign_idx]
        y_true_bin = (yte != benign_idx).astype(int)
        t1, f1, T1 = fpr_at_threshold(y_true_bin, attack_scores, 1e-3)
        t2, f2, T2 = fpr_at_threshold(y_true_bin, attack_scores, 1e-4)

        # Per-class metrics
        y_pred = proba_te.argmax(axis=1)
        perclass = []
        for cidx, cname in enumerate(FAMILIES):
            idxs = np.where(yte == cidx)[0]
            if len(idxs) == 0:
                rec_c = float('nan'); f1_c = float('nan'); sup_c = 0
            else:
                tp = np.sum(y_pred[idxs] == cidx)
                fn = len(idxs) - tp
                fp = np.sum((y_pred == cidx) & (yte != cidx))
                rec_c = tp / (tp + fn) if (tp + fn) > 0 else 0.0
                prec_c = tp / (tp + fp) if (tp + fp) > 0 else 0.0
                f1_c = 2*prec_c*rec_c/(prec_c+rec_c) if (prec_c+rec_c) > 0 else 0.0
                sup_c = len(idxs)
            perclass.append({"setting": f"{train_domain}->{test_domain}",
                             "model_name": model_kind,
                             "use_adv": use_adv,
                             "class": cname,
                             "recall": float(rec_c),
                             "f1": float(f1_c),
                             "support": int(sup_c)})
        perclass_dfs.append(pd.DataFrame(perclass))

        # Reliability bins
        rel_rows = []
        for (conf_mean, acc, count) in rel:
            rel_rows.append({"setting": f"{train_domain}->{test_domain}",
                             "model_name": model_kind,
                             "use_adv": use_adv,
                             "conf_mean": float(0.0 if (conf_mean!=conf_mean) else conf_mean),
                             "acc": float(0.0 if (acc!=acc) else acc),
                             "count": int(count)})
        relbin_dfs.append(pd.DataFrame(rel_rows))

        return RunResult(setting=f"{train_domain}->{test_domain}", model_name=model_kind, use_adv=use_adv,
                         resampling=CONFIG["RESAMPLING"], loss_mode=CONFIG["LOSS_MODE"],
                         macro_f1=float(macro_f1), macro_recall=float(macro_rec),
                         roc_auc_ovr=float(roc_ovr) if not np.isnan(roc_ovr) else np.nan,
                         pr_auc_ovr=float(pr_ovr) if not np.isnan(pr_ovr) else np.nan,
                         ece=float(ece), mce=float(mce),
                         fpr1e3=float(f1), tpr_at_fpr1e3=float(T1),
                         fpr1e4=float(f2), tpr_at_fpr1e4=float(T2))

    for model_kind in ["LR-only", "BiLSTM-only", "LR->BiLSTM"]:
        # In-domain
        results.append(run_setting("IoMT", "IoMT", model_kind, use_adv=False))
        results.append(run_setting("IoT",  "IoT",  model_kind, use_adv=False))
        # Cross-domain
        results.append(run_setting("IoMT", "IoT",  model_kind, use_adv=(model_kind!="LR-only")))
        results.append(run_setting("IoT",  "IoMT", model_kind, use_adv=(model_kind!="LR-only")))

    df = pd.DataFrame([asdict(r) for r in results])
    out_csv = os.path.join(CONFIG["OUTDIR"], "domain_shift_results_v2.csv")
    os.makedirs(CONFIG["OUTDIR"], exist_ok=True)
    df.to_csv(out_csv, index=False)

    out_pc = os.path.join(CONFIG["OUTDIR"], "domain_shift_perclass_v2.csv")
    pd.concat(perclass_dfs, ignore_index=True).to_csv(out_pc, index=False)

    out_rel = os.path.join(CONFIG["OUTDIR"], "domain_shift_reliability_bins_v2.csv")
    pd.concat(relbin_dfs, ignore_index=True).to_csv(out_rel, index=False)

    print("\\nSaved:", out_csv)
    print("\\nSaved:", out_pc)
    print("\\nSaved:", out_rel)
    print(df.to_string(index=False))

if __name__ == "__main__":
    main()

[WARN] MQTT filter produced empty set; reverting to full NF-ToN-IoT.
Epoch 1/30
6554/6554 - 47s - 7ms/step - accuracy: 0.9993 - loss: 0.0105 - val_accuracy: 0.8074 - val_loss: 1.8886
Epoch 2/30
6554/6554 - 42s - 6ms/step - accuracy: 1.0000 - loss: 5.5380e-04 - val_accuracy: 0.8074 - val_loss: 2.1057
Epoch 3/30
6554/6554 - 44s - 7ms/step - accuracy: 1.0000 - loss: 1.1029e-04 - val_accuracy: 0.8074 - val_loss: 2.4000
Epoch 4/30
6554/6554 - 44s - 7ms/step - accuracy: 1.0000 - loss: 4.3157e-06 - val_accuracy: 0.8074 - val_loss: 3.0364
Epoch 5/30
6554/6554 - 43s - 7ms/step - accuracy: 1.0000 - loss: 7.8187e-08 - val_accuracy: 0.8074 - val_loss: 3.3394
Epoch 6/30
6554/6554 - 44s - 7ms/step - accuracy: 1.0000 - loss: 5.0444e-11 - val_accuracy: 0.8074 - val_loss: 3.4623
Epoch 1/30
5243/5243 - 41s - 8ms/step - accuracy: 0.9985 - loss: 0.0387 - val_accuracy: 1.0000 - val_loss: 0.0080
Epoch 2/30
5243/5243 - 35s - 7ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 1.0000 - val_loss: 0.0042

In [58]:
import google.colab.files
import os

# Define the output directory from the CONFIG
output_dir = CONFIG["OUTDIR"]

# Define the filenames to download
result_files = [
    "domain_shift_results_v2.csv",
    "domain_shift_perclass_v2.csv",
    "domain_shift_reliability_bins_v2.csv"
]

print("Attempting to download result files...")

# Download each file
for filename in result_files:
    file_path = os.path.join(output_dir, filename)
    if os.path.exists(file_path):
        try:
            google.colab.files.download(file_path)
            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"File not found: {filename}")

print("Download process finished.")

Attempting to download result files...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: domain_shift_results_v2.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: domain_shift_perclass_v2.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: domain_shift_reliability_bins_v2.csv
Download process finished.
